<a href="https://colab.research.google.com/github/tshehadey/Projects/blob/main/Movie%20Review%20Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sentiment Analysis on IMDB Movie Reviews**

### **Project Overview**

In this project, we aim to perform sentiment analysis on movie reviews from the IMDB dataset. Sentiment analysis is a common task in natural language processing (NLP) where the goal is to classify the sentiment expressed in a piece of text. Specifically, we want to determine whether a given movie review expresses a positive or negative sentiment.

In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

In [ ]:
import tensorflow_datasets as tfds

# The dataset contains 50,000 movie reviews with their respective sentiment labels (positive/negative)
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

# Split the dataset into training and testing sets
train_data, test_data = dataset['train'], dataset['test']

In [ ]:
# Display the first few reviews and their corresponding labels
for review, label in train_data.take(5):
    print(f'Review: {review.numpy()}')
    print(f'Label: {"Positive" if label.numpy() == 1 else "Negative"}')
    print('---')

In [ ]:
# Calculate the distribution of labels in the training dataset
positive_reviews = 0
negative_reviews = 0

for _, label in train_data:
    if label.numpy() == 1:
        positive_reviews += 1
    else:
        negative_reviews += 1

print(f'Number of Positive Reviews: {positive_reviews}')
print(f'Number of Negative Reviews: {negative_reviews}')
print(f'Total Reviews: {positive_reviews + negative_reviews}')


In [ ]:
# Display the length of the first few reviews before padding
for review, _ in train_data.take(5):
    print(f'Review length (before padding): {len(review.numpy())}')

In [ ]:
import matplotlib.pyplot as plt

review_lengths = [len(review.numpy()) for review, _ in train_data]

plt.figure(figsize=(10, 6))
plt.hist(review_lengths, bins=20, color='green')
plt.title('Distribution of Review Lengths')
plt.xlabel('Length of Review')
plt.ylabel('Count')
plt.show()


In [ ]:
# Define constants
BUFFER_SIZE = 10000
BATCH_SIZE = 64
VOCAB_SIZE = 10000

# Build the tokenizer using the training data
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    (review.numpy() for review, label in train_data), target_vocab_size=VOCAB_SIZE)

def encode_review(review, label):
    encoded_review = tokenizer.encode(review.numpy())
    encoded_review = [min(token, VOCAB_SIZE - 1) for token in encoded_review]
    return encoded_review, label

def encode_map_fn(review, label):
    encoded_review, label = tf.py_function(encode_review, inp=[review, label], Tout=(tf.int64, tf.int64))
    encoded_review.set_shape([None])
    label.set_shape([])
    return encoded_review, label

# Apply the encoding function to the training and testing data
train_data = train_data.map(encode_map_fn)
test_data = test_data.map(encode_map_fn)

# Batch and pad the datasets to ensure each batch has the same shape
train_data = train_data.padded_batch(BATCH_SIZE, padded_shapes=([None], []))
test_data = test_data.padded_batch(BATCH_SIZE, padded_shapes=([None], []))

In [ ]:
# Create the model using Sequential API
model = tf.keras.Sequential([
    tf.keras.Input(shape=(100,)),
    # Embedding layer converts integer sequences to dense vector representations
    tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=64,),

    # Bidirectional LSTM layers to capture information from both directions
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),

    # Fully connected dense layer with ReLU activation
    tf.keras.layers.Dense(64, activation='relu'),

    # Output layer with sigmoid activation for binary classification
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model with binary crossentropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Print the model architecture
model.summary()

In [ ]:
# Train the model for 5 epochs and validate on the test data
history = model.fit(train_data, epochs=5, validation_data=test_data)

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Evaluate the model on the test data to get the accuracy
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Save the model in the recommended Keras format
model.save('movie_review.keras')

In [ ]:
from tensorflow.keras.models import load_model

# Load the model from the .keras file
model = load_model('movie_review.keras')

In [ ]:
y_pred = model.predict(test_data)

y_pred_classes = (y_pred > 0.5).astype("int32")

In [ ]:
# Extract true labels from test_data
y_true = []
for _, label in test_data:
    y_true.extend(label.numpy())

import numpy as np
# Convert to numpy array for compatibility with sklearn
y_true = np.array(y_true)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred_classes)
recall = recall_score(y_true, y_pred_classes)
f1 = f1_score(y_true, y_pred_classes)

# Print out the metrics
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)
cm

In [ ]:
# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# **Conclusion**

### **Model Performance**

In this project, we successfully built and trained a deep learning model to perform sentiment analysis on the IMDB movie reviews dataset. The model achieved a test accuracy of 86.63%, which indicates a strong ability to correctly classify movie reviews as either positive or negative.

### **Generalization to Unseen Data**

The model demonstrates solid performance on the test data, suggesting that it has learned meaningful patterns from the training data that generalize well to unseen reviews. However, there are still opportunities for improvement, especially in fine-tuning the model to enhance precision without compromising recall.

### **Potential Next Steps**

While the model generalizes well to unseen data, further improvements could be achieved through hyperparameter tuning, exploring alternative architectures like GRUs or attention mechanisms, applying data augmentation techniques to enhance diversity, and incorporating regularization methods to prevent overfitting.

### **Final Thoughts**

This project showcases the potential of deep learning models for text classification tasks such as sentiment analysis. With further refinement, the model can be adapted and improved to achieve even better performance, making it a valuable tool for automatically assessing the sentiment expressed in text data.